In [ ]:
#!pip install -q python-crfsuite
!pip install -q sklearn-crfsuite
!git clone https://github.com/simejisan/ML_Course_2020.git


     |████████████████████████████████| 747kB 3.8MB/s 
Cloning into 'ML_Course_2020'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 23 (delta 1), reused 0 (delta 0), pack-reused 15
Unpacking objects: 100% (23/23), done.


In [ ]:
!pip install pandarallel

  Created wheel for pandarallel: filename=pandarallel-1.4.8-cp36-none-any.whl size=16112 sha256=f618a0ecfcf006f18488cb2e5ff07e7a18d6a3d8b1948fda6bf8f9bc2228ae88
  Stored in directory: /root/.cache/pip/wheels/75/a2/85/b45be2e86d86e9ec5da6d05c4b994d18c81abe76e3f39415aa
Successfully built pandarallel


In [ ]:
from itertools import chain
import nltk
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer
import sklearn_crfsuite as crf
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics
from collections import Counter
import pandas as pd
import numpy as np
import re
import os.path
from itertools import chain
from parsing import symbols, tokenize, standardize, isquantity, isunit, asfloat, tokenmatch, round_2f, iobtag, removeiob




In [ ]:
url = 'https://raw.githubusercontent.com/simejisan/ML_Course_2020/master/nyt-ingredients-snapshot-2015.csv'
df = pd.read_csv(url)


In [ ]:
pd.read_csv('nyt-ingredients-snapshot-2015.csv', delimiter =',')

In [ ]:
df = df[:100000]
df.head()

,index,input,name,qty,range_end,unit,comment
0,0,1 1/4 cups cooked and pureed fresh butternut s...,butternut squash,1.25,0.0,cup,"cooked and pureed fresh, or 1 10-ounce package..."
1,1,1 cup peeled and cooked fresh chestnuts (about...,chestnuts,1.00,0.0,cup,"peeled and cooked fresh (about 20), or 1 cup c..."
2,2,"1 medium-size onion, peeled and chopped",onion,1.00,0.0,NaN,"medium-size, peeled and chopped"
3,3,"2 stalks celery, chopped coarse",celery,2.00,0.0,stalk,chopped coarse
4,4,1 1/2 tablespoons vegetable oil,vegetable oil,1.50,0.0,tablespoon,NaN


In [ ]:
df.isnull().sum()


index            0
input           63
name            32
qty              0
range_end        0
unit         26489
comment      35908
dtype: int64

In [27]:
from training import generatedata

X, y, X_test, y_test = generatedata("nyt-ingredients-snapshot-2015.csv", testprop=0.2, parallel=True)

In [ ]:
#Training
%%time
from training import trainCRF

filename = 'model.crfsuite'
trainCRF(X, y, output=filename)

In [ ]:
#Training Data Evaluation

from evaluate import evaluate

accuracy, precision, recall, fscore = evaluate(X, y, filename)

print("Accuracy:")
print(accuracy)
print("Precision:")
print(precision)
print("Recall:")
print(recall)
print("F-Score")
print(fscore)

Accuracy:
{'INGR': 0.8959051306351907, 'QTY': 0.9390052116510351, 'QTY-UR': 0.9249659824231292, 'UNIT': 0.9345101178290259, 'Total': 0.9157275959013917}
Precision:
{'INGR': 0.9177369744206652, 'QTY': 0.9813806868817806, 'QTY-UR': 0.541958041958042, 'UNIT': 0.922226841266542, 'Total': 0.9333623258799897}
Recall:
{'INGR': 0.8517239650360169, 'QTY': 0.9825457924077515, 'QTY-UR': 0.7332939089296274, 'UNIT': 0.9701353838304696, 'Total': 0.9076304853266939}
F-Score
{'INGR': 0.8834991010920389, 'QTY': 0.9819628940435174, 'QTY-UR': 0.623272178939432, 'UNIT': 0.9455746672231449, 'Total': 0.9203165766506621}


In [ ]:
#Test Set Evaluation

accuracy, precision, recall, fscore = evaluate(X_test, y_test, filename)

print("Accuracy:")
print(accuracy)
print("Precision:")
print(precision)
print("Recall:")
print(recall)
print("F-Score")
print(fscore)


Accuracy:
{'INGR': 0.888933313947853, 'QTY': 0.9323822976979557, 'QTY-UR': 0.9160440980495094, 'UNIT': 0.92747700073272, 'Total': 0.9103946566205309}
Precision:
{'INGR': 0.9066589312189777, 'QTY': 0.9798427070253123, 'QTY-UR': 0.4795539033457249, 'UNIT': 0.920094191522763, 'Total': 0.9263259304330189}
Recall:
{'INGR': 0.8472636366374506, 'QTY': 0.9827654779265544, 'QTY-UR': 0.675392670157068, 'UNIT': 0.970364238410596, 'Total': 0.9050331913207174}
F-Score
{'INGR': 0.875955594998988, 'QTY': 0.9813019161399213, 'QTY-UR': 0.5608695652173913, 'UNIT': 0.9445608380338437, 'Total': 0.9155557782900671}


In [ ]:
#Sample tagging
import pandas as pd
from parsing import tokenize, removeiob
from evaluate import getlabels

def displaytags(tokens, tags):
    # Make a table with pandas and transpose to make horizontal
    df = pd.DataFrame(tokens, removeiob(pred)).transpose()
    # Print string representation with adjusted spacing and display options
    print(df.to_string(index=False, justify='center', col_space=8, max_cols=15))

df = pd.read_csv('nyt-ingredients-snapshot-2015.csv')
df = df.loc[pd.notna(df.name)&pd.notna(df.input)]

In [ ]:
from numpy.random import randint
samples = df.input.iloc[randint(0, len(df), 10)]
    
for item in samples:
    tokens = tokenize(item, preprocess=True)
    pred = getlabels(item, filename)
    displaytags(tokens, pred)
    print('-'*100)

  QTY       UNIT                       INGR     INGR     INGR     INGR     INGR     INGR  
   1/2    teaspoon  freshly  ground    black   pepper      ,      more      to      taste 
----------------------------------------------------------------------------------------------------
           INGR     INGR     INGR     INGR     INGR  
 Grated   Cheddar  cheese      ,       for    garnish
----------------------------------------------------------------------------------------------------
  QTY      UNIT     INGR  
    1       cup     sugar 
----------------------------------------------------------------------------------------------------
  QTY                        INGR     INGR     INGR     INGR     INGR     INGR     INGR     INGR     INGR  
    1       to        2     garlic   cloves      ,       to      taste      ,     minced     or     puréed 
----------------------------------------------------------------------------------------------------
  QTY       INGR                    

In [ ]:
recipe = ['1 14-oz. package firm or extra-firm tofu, drained',
'1 Tbsp. black peppercorns',
'2 garlic cloves',
'1 1½" piece ginger, peeled',
'1 Tbsp. cornstarch',
'½ tsp. kosher salt',
'3 Tbsp. extra-virgin olive oil',
'1 lb. asparagus, trimmed, cut into 1½" pieces',
'⅓ cup soy sauce',
'1 Tbsp. sugar',
'1 tsp. unseasoned rice vinegar',
'Cooked white or brown rice (for serving)']

for item in recipe:
    tokens = tokenize(item, preprocess=True)
    pred = getlabels(item, filename)
    displaytags(tokens, pred)
    print('-'*100)

  QTY                                                               INGR     INGR     INGR  
    1      14-oz      .     package   firm      or     extra-firm   tofu       ,     drained
----------------------------------------------------------------------------------------------------
  QTY      UNIT              INGR       INGR    
    1      Tbsp       .      black   peppercorns
----------------------------------------------------------------------------------------------------
  QTY      INGR     INGR  
    2     garlic   cloves 
----------------------------------------------------------------------------------------------------
  QTY                        UNIT     INGR     INGR     INGR  
    1      1$1/2      "      piece   ginger      ,     peeled 
----------------------------------------------------------------------------------------------------
  QTY      UNIT                INGR   
    1      Tbsp       .     cornstarch
------------------------------------------------------

In [ ]:
print('Enter a single recipe line to be tagged or type EXIT to stop')
while True:
    
    s = input('')
    if s.lower() == 'exit': break
    
    tokens = tokenize(s, preprocess=True)
    pred = getlabels(s, filename)
    print('')
    displaytags(tokens, pred)
    print('')
    print('-'*100)

Enter a single recipe line to be tagged or type EXIT to stop


KeyboardInterrupt: ignored